# Lab 1 - Overview of embeddings-based retrieval

Welcome! Here's a few notes about the Chroma course notebooks.

A number of warnings pop up when running the notebooks. These are normal and can be ignored.
Some operations such as calling an LLM or an opeation using generated data return unpredictable results and so your notebook outputs may differ from the video.
Enjoy the course

In [37]:

import PyPDF2
reader = PyPDF2.PdfReader('data/001_ADZYNMA.pdf')

text_divided_by_pages = []

# printing number of pages in pdf file
text =''
for j in range(len(reader.pages)):

    # getting a specific page from the pdf file
    page = reader.pages[j]

    # extracting text from page
    text = text + page.extract_text()
    #text = unidecode.unidecode(text)   

    #Appending pages in to the list, we need to have the pages separated
    text_divided_by_pages.append(page.extract_text())


In [42]:
#This function will help us to remove all blank spaces and empy pages..

def processing_blank_spaces_in_pages(page):
    page = page.replace('\n','')
    page = page.replace('  ','')
    return page

In [43]:
for i in range(len(text_divided_by_pages)):

    text_divided_by_pages[i] = processing_blank_spaces_in_pages(text_divided_by_pages[i])

In [45]:
text_divided_by_pages[1]

'2 Este medicamento está sujeto a seguimiento adicional, lo que agilizará la detección de nueva información sobre su seguridad. Se invita a los profesionales sanitarios a notificar las sospechas de reacciones adversas. Ver la sección 4.8, en la que se incluye información sobre cómo notificarlas. 1. NOMBRE DEL MEDICAMENTOADZYNMA 500 UI polvo y disolvente para solución inyectable ADZYNMA 1 500 UI polvo y disolvente para solución inyectable 2. COMPOSICIÓN CUALITATIVA Y CUANTITATIVAADZYNMA 500 UI polvo y disolvente para solución inyectableCada vial de polvo contiene nominalmente 500 unidades internacionales (UI) de actividad rADAMTS13*, medidas en términos de potencia.Tras su reconstitución en los 5 ml de disolvente proporcionados, la solución tiene una potencia de aproximadamente 100 UI/ml.ADZYNMA 1 500 UI polvo y disolvente para solución inyectableCada vial de polvo contiene nominalmente 1 500 unidades internacionales (UI) de actividad rADAMTS13*, medidas en términos de potencia.Tras su 

In [35]:
x=text.replace('\n','')
x=x.replace('  ','')
x

'1ANEXO IFICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO 2 Este medicamento está sujeto a seguimiento adicional, lo que agilizará la detección de nueva información sobre su seguridad. Se invita a los profesionales sanitarios a notificar las sospechas de reacciones adversas. Ver la sección 4.8, en la que se incluye información sobre cómo notificarlas. 1. NOMBRE DEL MEDICAMENTOADZYNMA 500 UI polvo y disolvente para solución inyectable ADZYNMA 1 500 UI polvo y disolvente para solución inyectable 2. COMPOSICIÓN CUALITATIVA Y CUANTITATIVAADZYNMA 500 UI polvo y disolvente para solución inyectableCada vial de polvo contiene nominalmente 500 unidades internacionales (UI) de actividad rADAMTS13*, medidas en términos de potencia.Tras su reconstitución en los 5 ml de disolvente proporcionados, la solución tiene una potencia de aproximadamente 100 UI/ml.ADZYNMA 1 500 UI polvo y disolvente para solución inyectableCada vial de polvo contiene nominalmente 1 500 unidades internacionales (U

Now we have to chunck up the text, by character....

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=10,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(text_divided_by_pages[2]))

print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}")

i

s

t

Total chunks: 779


In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

In [ ]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

In [ ]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))